In [131]:
import graphlab
import os
from graphlab import mxnet as mx
#Limitacion de Hilos
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)
#Descarga y guardar el modelo iter45
pretrained_model_iter_45=graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
pretrained_model_iter_45.save('imagenet_model_iter45')
#Carga inception_1k_v3 (https://github.com/dmlc/mxnet-model-gallery/blob/master/imagenet-1k-inception-v3.md)
pretrained_model_inceptionv3=mx.pretrained_model.load_path('F:\\MSc Misis\\4 Semestre\\Tesis 1\\Modelos\\imagenet1k_inception_v3')
#Carga inception_21k (https://github.com/dmlc/mxnet-model-gallery/blob/master/imagenet-21k-inception.md)
pretrained_model_inception21k=mx.pretrained_model.load_path('F:\\MSc Misis\\4 Semestre\\Tesis 1\\Modelos\\imagenet21k_inception_bn')

In [98]:

#creación de SArray para introducir columna de label en Training set
def Label_Training_column_set(num_images_Class1,num_label_Class1):
    a=[]
    b=[num_label_Class1]
    i=0
    if(i<num_images_Class1):
        while (i<num_images_Class1):
            a.extend(b)
            i=i+1
    label_array=graphlab.SArray(data=a, dtype=int)
    return label_array



#Función Preprocesamiento del Set de Datos de Entrenamiento
def Preproc_set(path, number_images_Class, num_label_Class):
    Path_images=path 
    image_sframe = graphlab.image_analysis.load_images(Path_images, "jpg", with_path=True,recursive=True)
    image_sframe['resized']=graphlab.image_analysis.resize(image_sframe['image'], 500, 500, 1)
    label_array=Label_Training_column_set(number_images_Class,num_label_Class )
    image_sframe.add_column(label_array, name='label')
    return image_sframe
              
#Carga de los datos
TS1= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Final_Moto",57,1)
TS2= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Final_No_Moto",52,0)
TS1=TS1.append(TS2)

Test_d1= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Test_Moto",16,1)
Test_d2= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Test_No_Moto",19,0)
Test_d1=Test_d1.append(Test_d2)


               



In [47]:
#Carga de modelos
pretrained_model_iter_45=graphlab.load_model('imagenet_model_iter45')


Class               : NeuralNetClassifier

Schema
------
Examples            : 1281166
Features            : 1
Target column       : label

Training Summary
----------------
Training accuracy   : 0.625
Validation accuracy : None
Training recall@5   : 0.8512
Validation recall@5 : None
Training time (sec) : 288884.0298

In [132]:
#Extracción de Caracerísticas de DataSet de Entrenamiento y Prueba con modelos iter_45, inceptionv3, inception21k
TS1['features_iter_45']= pretrained_model_iter_45.extract_features(TS1[['image']])
TS1['features_inceptionv3']= pretrained_model_inceptionv3.extract_features(TS1[['image']])
TS1['features_inception21K']= pretrained_model_inception21k.extract_features(TS1[['image']])
Test_d1['features_iter_45']= pretrained_model_iter_45.extract_features(Test_d1[['image']])
Test_d1['features_inceptionv3']= pretrained_model_inceptionv3.extract_features(Test_d1[['image']])
Test_d1['features_inception21K']= pretrained_model_inception21k.extract_features(Test_d1[['image']])
TS1.save('TS.sframe')
Test_d1.save('Test_d.sframe')

Images being resized.

[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (299, 299, 3)
[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (224, 224, 3)


Images being resized.

[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (299, 299, 3)
[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (224, 224, 3)


In [153]:
#Creacion de modelo KNN para crear patrón de similaridad
#modelo_bike_nn=graphlab.nearest_neighbors.create(TS1, features=['features'])
# División de Datos
training_data, validation_data = TS1.random_split(0.9)
#Creación de Clasificadores
#bike_classifier_model_NN = graphlab.neuralnet_classifier.create(training_data, target='label',  features=['features'],
                                                                #validation_set=validation_data)
bike_classifier_model_BTC=graphlab.boosted_trees_classifier.create(training_data, target='label', features=['features_iter_45', 
                                                                                                            'features_inceptionv3',
                                                                                                           'features_inception21K'],
                                                                   max_iterations=10, validation_set=validation_data,
                                                                  max_depth=5, step_size=0.5, metric='accuracy', 
                                                                  early_stopping_rounds=3)
#bike_classifier_model_RF= graphlab.random_forest_classifier.create(training_data, target='label', features=['features'],
                                                                   #validation_set=validation_data)
bike_classifier_model_DT= graphlab.decision_tree_classifier.create(training_data, target='label', features=['features_iter_45', 
                                                                                                            'features_inceptionv3',
                                                                                                           'features_inception21K'],
                                                                   validation_set=validation_data)
#bike_classifier_model_LR= graphlab.logistic_classifier.create(training_data, target='label', features=['features_iter_45', 
                                                                                                            #'features_inceptionv3',
                                                                                                           #'features_inception21K'],
                                                                   #validation_set=validation_data)
#bike_classifier_model_SVM= graphlab.svm_classifier.create(training_data, target='label', features=['features_iter_45',
                                                                                                    #'features_inceptionv3',
                                                                                                    #'features_inception21K'],
                                                                   #solver='lbfgs',max_iterations=8,validation_set=validation_data)


#Clasificación del Set de Prueba
#results_NN=bike_classifier_model_NN.evaluate(Test_d1)
results_BTC=bike_classifier_model_BTC.evaluate(Test_d1)
#results_RF=bike_classifier_model_RF.evaluate(Test_d1)
results_DT=bike_classifier_model_DT.evaluate(Test_d1)
#results_LR=bike_classifier_model_LR.evaluate(Test_d1)
results_SVM=bike_classifier_model_SVM.evaluate(Test_d1)


WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'features_iter_45', 'features_inception21K' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 100

...

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| Iteration | Elapsed Time | Training-accuracy | Validation-accuracy | Training-log_loss | Validation-log_loss |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

| 1         | 0.092060     | 1.000000          | 1.000000            | 0.456655          | 0.453863            |

+-----------+--------------+-------------------+---------------------+-------------------+---------------------+

In [156]:
results_BTC


{'accuracy': 0.9714285714285714, 'auc': 0.96875, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 3
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   1   |
 |      1       |        1        |   15  |
 |      0       |        0        |   19  |
 +--------------+-----------------+-------+
 [3 rows x 3 columns], 'f1_score': 0.967741935483871, 'log_loss': 0.3665163140328247, 'precision': 1.0, 'recall': 0.9375, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 16 | 19 |
 |   1e-05   | 1.0 | 1.0 | 16 | 19 |
 |   2e-05   | 1.0 | 1.0 | 16 | 19 |
 |   3e-05   | 1.0 | 1.0 | 16 | 19 |
 |   4e-05   | 1.0 | 1.0 | 16 | 19 |
 |   5e-05   | 1.0